Inspeccion inicial, La secuencia empieza en el libro 1

In [1]:
import numpy as np
import pandas as pd
import time
import pyvo
import warnings
import astropy.units as u
import matplotlib.pyplot as plt
from datetime import date
from astropy.table import Table
from astropy.table import hstack
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord
from matplotlib import pyplot as plt
from astropy import coordinates, units as u
from astroquery.xmatch import XMatch

Could not import regions, which is required for some of the functionalities of this module.


In [2]:
query = """SELECT TOP 500000
    s.specobjid, s.ra, s.dec, s.z, g.h_alpha_flux, g.h_beta_flux,
    g.oiii_5007_flux, g.nii_6548_flux, g.nii_6584_flux,
    g.sii_6717_flux, g.sii_6731_flux, g.oi_6300_flux, s.class, s.subclass

    FROM SpecObjAll AS s
    JOIN galSpecExtra AS ID ON ID.specObjID = s.specObjID
    JOIN galSpecLine AS g ON g.specObjID = s.specObjID


    WHERE subclass IN ('STARFORMING', 'STARBURST', 'AGN', 'BROADLINE')
    AND NOT h_alpha_flux <='0' AND NOT h_beta_flux <='0'
    AND NOT oiii_5007_flux <='0' AND NOT nii_6548_flux <='0'
    AND NOT nii_6584_flux <='0' AND NOT sii_6717_flux <='0' 
    AND NOT sii_6731_flux <='0' AND NOT oi_6300_flux <='0'
    AND z < 0.8

    ORDER BY specobjid ASC"""

SDSS = SDSS.query_sql(query)

C:\ProgramData\Anaconda33\lib\site-packages\astroquery\sdss\core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),


In [3]:
SDSS[0:5]

specobjid,ra,dec,z,h_alpha_flux,h_beta_flux,oiii_5007_flux,nii_6548_flux,nii_6584_flux,sii_6717_flux,sii_6731_flux,oi_6300_flux,class,subclass
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes6,bytes11
299489677444933632,146.71421,-1.0413043,0.02122228,472.5904,119.4956,68.39606,74.30252,224.1136,124.7115,84.18053,18.59493,GALAXY,STARFORMING
299490502078654464,146.62857,-0.76513683,0.06465632,99.40005,25.64047,21.44437,17.10464,51.59155,35.33497,21.65778,8.490424,GALAXY,STARFORMING
299491051364706304,146.9639,-0.54500256,0.05598787,181.5262,47.3665,11.85567,17.94872,54.13749,31.28298,19.16762,4.319232,GALAXY,STARBURST
299491051834468352,146.63167,-0.98827781,0.05265425,127.3058,38.33226,35.46671,8.550285,25.78963,30.26996,25.72297,6.269853,GALAXY,STARFORMING
299492700632147968,146.59272,-0.76025604,0.06502353,211.5018,53.18228,15.59901,25.31465,76.35482,41.6377,28.47964,7.211103,GALAXY,STARFORMING


In [35]:
len(SDSS)

329650

In [4]:
service = pyvo.dal.TAPService ('https://irsa.ipac.caltech.edu/TAP')
result = service.run_async("""
            SELECT
            galname,
            ra,
            dec,
            fnu_12,
            fnu_25,
            fnu_60,
            fnu_100,
            rat_12_25,
            rat_25_60,
            rat_60_100
            
            FROM irasgal
            ORDER BY galname ASC""")
IRAS = result.to_table()

In [5]:
IRAS[0:5]

galname,ra,dec,fnu_12,fnu_25,fnu_60,fnu_100,rat_12_25,rat_25_60,rat_60_100
,degrees,degrees,Jy,Jy,Jy,Jy,,,
object,float32,float32,float32,float32,float32,float32,float32,float32,float32
00000-0359,0.6448,-3.7105,0.25,0.33,1.21,2.49,0.7576,0.2727,0.4859
00001+0827,0.6663,8.7373,0.25,0.28,0.70,1.19,0.8929,0.4000,0.5882
00003+0313,0.7196,3.5042,0.25,0.32,0.52,1.12,0.7812,0.6154,0.4643
00003-3431,0.7281,-34.2385,1.06,0.25,0.80,2.57,4.2400,0.3125,0.3113
00005+2140,0.7878,21.9597,0.25,1.05,4.25,4.51,0.2381,0.2471,0.9424


---

Verificamos la longitud de los dos catalogos. El catalogo que tenga mas elementos será el que utilizaremos como referencia.


In [6]:
len(IRAS), len(SDSS)

(11444, 329650)

---

Ahora crearemos cuatro arreglos de `NumPy` con la información de las coordenadas celestes

In [7]:
ra1 = np.asarray(IRAS['ra'])
dec1 = np.asarray(IRAS['dec'])
ra2 = np.asarray(SDSS['ra'])
dec2 = np.asarray(SDSS['dec'])

Utilizando esos arreglos, definiremos dos 'catalogos' utilizando la función SkyCoord( ). Nótese que las coordenadas celestes ya se plantean en grados dentro de los catalogos originales 

In [8]:
IRAScat = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)
SDSScat = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)

In [9]:
#Explora:
# type(IRAScat)
IRAScat
# cantidad_parejas = len(IRAScat)
# cantidad_parejas

<SkyCoord (ICRS): (ra, dec) in deg
    [(0.6447522 ,  -3.710484 ), (0.66634995,   8.73727  ),
     (0.7195514 ,   3.5042214), ..., (0.5100675 , -33.46959  ),
     (0.5424071 ,   2.942838 ), (0.57888216,  12.971159 )]>

In [10]:
#Explora:
# type(SDSScat)
# SDSScat
cantidad_parejasSD = len(SDSScat)
cantidad_parejasSD

329650

Ahora realizamos el crossmatching mediante el método .match_to_catalog( )

En este caso utilizaremos el catálogo SDSS como referencia y lo cruzamos con el catalogo IRAS.

El método retorna tres resultados:

- idx : indices dentro del catálogo IRAS que corresponden a los objetos mas cercanos a cada uno de los objetos del catálogo SDSS
- d2d : distancias en el cielo (2-dimensionales) entro los objetos que hacen match.
- d3d : distancias 3-dimensionales (al no haber dato 3D, se asume que todos los objetos estan a una distancia de 1 del observador)

In [11]:
idx, d2d, d3d = SDSScat.match_to_catalog_sky(IRAScat)

El tamaño del arreglo `idx` es igual al del catálogo SDSS:

In [12]:
idx.shape

(329650,)

---

Ahora podemos ver algunos de los match. Por ejemplo, para el objeto con índice **10719** del SDSS se tiene:

In [13]:
idx[10719]

8451

In [14]:
SDSS[10719]

specobjid,ra,dec,z,h_alpha_flux,h_beta_flux,oiii_5007_flux,nii_6548_flux,nii_6584_flux,sii_6717_flux,sii_6731_flux,oi_6300_flux,class,subclass
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes6,bytes11
354807207503620096,235.4981,0.71280211,0.006394573,150.4484,30.57159,12.23558,14.36272,43.32129,42.70899,28.75874,11.33575,GALAXY,STARFORMING


In [15]:
IRAS[idx[10719]]

galname,ra,dec,fnu_12,fnu_25,fnu_60,fnu_100,rat_12_25,rat_25_60,rat_60_100
,degrees,degrees,Jy,Jy,Jy,Jy,,,
object,float32,float32,float32,float32,float32,float32,float32,float32,float32
15394+0052,235.4980,0.7127,0.34,0.26,0.43,1.41,1.3077,0.6047,0.3050


La distancia entre estos dos objetos es

In [16]:
d2d[10719]

<Angle 0.00015679 deg>

In [17]:
# Explora:

# type(idx)
# len(idx)
# idx[1520:1530]

In [18]:
# Explora:

# type(idx)
# len(d2d)
d2d[1520:1530]

<Angle [1.19021246, 1.00341829, 1.01220856, 1.09484721, 1.2856854 ,
        1.32240247, 1.37393583, 1.46307341, 1.27553726, 1.47765342] deg>

In [19]:
# Explora:

# type(d3d)
# len(d3d)
d3d[1520:1530]

<Quantity [0.02077266, 0.01751265, 0.01766606, 0.0191083 , 0.0224389 ,
           0.0230798 , 0.02397905, 0.02553479, 0.02226179, 0.02578924]>

---

Debido a que los dos catalogos tienen diferente tamaño, es posible que varios elementos del catalogo SDSS tengan match con un mimso objeto del catalogo IRAS.

Para revisar esta posibilidad, debemos identificar los valores únicos dentro del match. Esto lo hacemos con la función [np.unique( )](https://numpy.org/doc/stable/reference/generated/numpy.unique.html). En esta función utilizaremos el argumento `return_counts=True` para saber cuantas veces se repite un mismo valor.

In [20]:
unique, c = np.unique(idx, return_counts=True)

Los valores únicos son:

In [21]:
unique

array([    5,     9,    17, ..., 11439, 11440, 11443], dtype=int64)

In [22]:
# Explora
len(unique)

3727

El número de repeticiones de cada elemento es

In [23]:
c

array([ 27, 339, 123, ...,   1,   1,   9], dtype=int64)

In [24]:
len(c)

3727

Esto queire decir que:

- El índice 5 se repite 5 veces
- el índice 17 se repite 20 veces
- el índice 42 se repite 93 veces
- el índice 84 se repite 8 veces
- ...

Para saber en donde se repite el cada índice, utilizamos la función [np.where( )](https://numpy.org/doc/stable/reference/generated/numpy.unique.html).

Por ejemplo para el índice 5 se tiene

In [25]:
np.where(idx==5)

(array([ 19711,  19720,  19721,  19722,  19724,  71678,  71691, 135496,
        135498, 135499, 135500, 181407, 181408, 181409, 181410, 181413,
        181418, 181419, 181420, 184847, 184861, 184862, 184863, 184865,
        184884, 184885, 184895], dtype=int64),)

In [26]:
indices = np.where(idx==5)[0]
print(f"Match IRAS[{5}] : ra = {IRAS['ra'][5]} , dec = {IRAS['dec'][5]}" )
for i in indices:
    print(f"SDSS[{i}] : ra = {SDSS['ra'][i]} , dec = {SDSS['dec'][i]}, d2d = {d2d[i]}" )

Match IRAS[5] : ra = 0.7726925015449524 , dec = -1.9154912233352661
SDSS[19711] : ra = 0.51393115 , dec = -1.2063099, d2d = 0.7548806612597737 deg
SDSS[19720] : ra = 0.023121653 , dec = -1.2163687, d2d = 1.0247936214533742 deg
SDSS[19721] : ra = 0.056005303 , dec = -1.2136299, d2d = 1.002925864629819 deg
SDSS[19722] : ra = 0.12870372 , dec = -1.2130045, d2d = 0.9528350715932876 deg
SDSS[19724] : ra = 0.16654755 , dec = -1.1911967, d2d = 0.9443194044874836 deg
SDSS[71678] : ra = 0.51393115 , dec = -1.2063099, d2d = 0.7548806612597737 deg
SDSS[71691] : ra = 0.24594648 , dec = -1.1079573, d2d = 0.9640406808048698 deg
SDSS[135496] : ra = 0.302984 , dec = -0.999136, d2d = 1.029652980266088 deg
SDSS[135498] : ra = 0.036677 , dec = -1.238461, d2d = 0.9998359551713715 deg
SDSS[135499] : ra = 0.2637187 , dec = -0.87462349, d2d = 1.1585762572417622 deg
SDSS[135500] : ra = 0.20495329 , dec = -1.03405, d2d = 1.0483540826732913 deg
SDSS[181407] : ra = 0.230996 , dec = -1.139509, d2d = 0.94624006132

In [27]:
# Explora:

np.where(idx==84)

(array([ 19895,  19896,  19897,  19898,  19899,  19900,  19916,  19917,
         20034,  20043,  20052,  20053,  20054,  20055,  20056,  20057,
         20060,  20061,  20063,  20064,  20065,  20066,  20067,  20068,
         20069,  20070,  20071,  20072,  20073,  20074,  20075,  20076,
         20077,  20078,  20079,  20080,  20081,  20106,  20120,  20121,
         20122,  20123,  20124,  20125,  20126,  20127,  20128,  20129,
         20130,  20132,  20133,  20134,  20135,  20136,  20137,  20138,
         20139,  20140,  20141,  20142,  20143,  20144,  20145,  20146,
         20147,  20148,  20149,  20150,  20151,  20153,  20154,  20155,
         20156,  20157,  20158,  20159,  20160,  20161,  20162,  20169,
         20170,  20171,  20172,  20173,  20174,  20175,  20195,  20196,
         20197,  20199,  20200,  20202,  20203,  20204,  20205,  20206,
         20207,  20208,  20209,  20210,  20211,  20212,  20213,  20214,
         20215,  20216,  20217,  20218,  20219,  20220,  20221, 

In [28]:
# Explora:
indices = np.where(idx==84)[0]
print(f"Match IRAS[{84}] : ra = {IRAS['ra'][84]} , dec = {IRAS['dec'][84]}" )
for i in indices:
    print(f"SDSS[{i}] : ra = {SDSS['ra'][i]} , dec = {SDSS['dec'][i]}, d2d = {d2d[i]}" )

Match IRAS[84] : ra = 3.7286314964294434 , dec = 0.251241534948349
SDSS[19895] : ra = 3.1436066 , dec = 0.049789973, d2d = 0.6187359200563349 deg
SDSS[19896] : ra = 3.0743046 , dec = -0.10294555, d2d = 0.7440363500793212 deg
SDSS[19897] : ra = 3.0787286 , dec = -0.013361758, d2d = 0.7017022852215498 deg
SDSS[19898] : ra = 3.2319399 , dec = 0.051845141, d2d = 0.535219099463444 deg
SDSS[19899] : ra = 3.1669489 , dec = -0.036205308, d2d = 0.6309605762197946 deg
SDSS[19900] : ra = 3.2771615 , dec = -0.13841074, d2d = 0.5963665573464648 deg
SDSS[19916] : ra = 2.8606477 , dec = 0.067535306, d2d = 0.8872076142019346 deg
SDSS[19917] : ra = 2.9560652 , dec = -0.031760269, d2d = 0.8227668473269437 deg
SDSS[20034] : ra = 2.4377834 , dec = 0.58299646, d2d = 1.3327629576446316 deg
SDSS[20043] : ra = 2.0330364 , dec = 1.2286852, d2d = 1.9570103303991677 deg
SDSS[20052] : ra = 2.8021843 , dec = 0.65141226, d2d = 1.0091502521893205 deg
SDSS[20053] : ra = 2.7823719 , dec = 0.59765268, d2d = 1.007648748

**La pregunta que surge es ¿Cual es el match apropiado?**

La respuesta esta en el investigador, quien debe elegir (de acuerdo con sus criterios) cual es la mejor opción. 

Por ejemplo, en este caso se podría decir que el mejor match es entre el SDSS[19711] y el IRAS[5] porque la distancia entre estos objetos es de 0.75488 deg.

También puede ser posible que el investigador considere los tres objetos mas cercanos en el SDSS correspondan a un solo objeto en el IRAS. Es decir que  SDSS[19711], SDSS[19722] y SDSS[19724] sean el objeto IRAS[5].



---

Una posibilidad adicional es restringir los resultados con una distancía máxima dentro de la cual se considera que si existe un match. Por ejemplo, podemos imponer que solamente si la distancia es menor que 10 arcsec se considere la posibilidad de match. 

In [29]:
max_sep = 10 * u.arcsec # Criterio de maxima distancia posible

# Creamos una mascara booleana
sep_constraint = d2d < max_sep
sep_constraint

array([False, False, False, ..., False, False, False])

In [30]:
SDSSmasked = SDSScat[sep_constraint]
IRASmasked = IRAScat[idx[sep_constraint]]

In [31]:
SDSSmasked

<SkyCoord (ICRS): (ra, dec) in deg
    [(145.51241,  0.33917087), (145.51241,  0.33917087),
     (167.2324 ,  0.44480423), ..., (242.24222, 19.823751  ),
     (245.78193, 19.004866  ), (258.72922, 35.520481  )]>

In [32]:
# Explora:

# type(SDSSmasked)
len(SDSSmasked)

1349

In [33]:
IRASmasked

<SkyCoord (ICRS): (ra, dec) in deg
    [(145.51395,  0.3375216 ), (145.51395,  0.3375216 ),
     (167.2346 ,  0.44466087), ..., (242.24101, 19.82251   ),
     (245.78319, 19.005993  ), (258.7292 , 35.520664  )]>

Asi se tienen solamente los objetos que hacen match y que cumplen con la ondiciónd e distancia máxima.

In [34]:
len(IRASmasked)

1349